In [ ]:
import requests
import pandas as pd
import numpy as np
import io
from time import sleep
from datetime import date
from datetime import datetime
from datetime import datetime
import os



def cp(ticker,key):
    #url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={ticker}&apikey={}'
    url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=IBM&outputsize=full&apikey=demo' # for demo data
    r = requests.get(url)
    data = r.json()
    # Extract the time series data from the response JSON
    time_series_data = data['Time Series (Daily)']
    # Convert to DataFrame
    df = pd.DataFrame(time_series_data).T
    # Reset index and rename columns
    df.reset_index(inplace=True)
    df.rename(columns={'index': 'Date'}, inplace=True)
    # Create a new column for the ticker symbol
    df['Ticker'] = ticker
    cols = list(df.columns)
    cols = ['Ticker'] + [col for col in cols if col != 'Ticker']
    df = df[cols]
    # Convert 'close' column to numeric
    #df['close'] = pd.to_numeric(df['4. close'])
    # Reverse the order of rows in the DataFrame
    #df = df[::-1].reset_index(drop=True)
    # Shift 'close' column by one row up to get tomorrow's close price
   # df['tomorrow_close'] = df['close'].shift(-1)
    #creating movement variable
    #df['movement'] = np.where(df['tomorrow_close'] > df['close'], 1, np.where(df['tomorrow_close'] < df['close'], -1, 0))
    
    return df.head(1000)

def rsi(ticker,key ):
    url = f'https://www.alphavantage.co/query?function=RSI&symbol={ticker}&interval=daily&time_period=10&series_type=open&apikey={key}' #changed to daily
    r = requests.get(url)
    data = r.json()
    # Extract the time series data from the response JSON
    time_series_data = data['Technical Analysis: RSI']
    # Convert to DataFrame
    df = pd.DataFrame(time_series_data).T
    # Reset index and rename columns
    df.reset_index(inplace=True)
    df.rename(columns={'index': 'Date'}, inplace=True)
    # Create a new column for the ticker symbol
    df['Ticker'] = ticker
    cols = list(df.columns)
    cols = ['Ticker'] + [col for col in cols if col != 'Ticker']
    df = df[cols]
    return df.head(1000)

def sma(ticker,key):
    url = f'https://www.alphavantage.co/query?function=SMA&symbol=IBM&interval=daily&time_period=10&series_type=open&apikey={key}' #changed to daily
    r = requests.get(url)
    data = r.json()
    # Extract the time series data from the response JSON
    time_series_data = data["Technical Analysis: SMA"]
    # Convert to DataFrame
    df = pd.DataFrame(time_series_data).T
    # Reset index and rename columns
    df.reset_index(inplace=True)
    df.rename(columns={'index': 'Date'}, inplace=True)
    # Create a new column for the ticker symbol
    df['Ticker'] = ticker
    cols = list(df.columns)
    cols = ['Ticker'] + [col for col in cols if col != 'Ticker']
    df = df[cols]
    return df.head(1000)


def eps(ticker,key):
    url = f'https://www.alphavantage.co/query?function=EARNINGS&symbol={ticker}&apikey={key}'
    r = requests.get(url)
    data = r.json()
    time_series_data = data["quarterlyEarnings"]
    # Convert to DataFrame
    df = pd.DataFrame(time_series_data).T
    # Reset index and rename columns
    df.reset_index(inplace=True)
    df.rename(columns={'index': 'Date'}, inplace=True)
    return df

def dty(key):
    url = f'https://www.alphavantage.co/query?function=TREASURY_YIELD&interval=daily&maturity=10year&apikey={key}' #changed to daily from default
    r = requests.get(url)
    data = r.json()
    time_series_data = data["data"]
    # Convert to DataFrame
    df = pd.DataFrame(time_series_data).T
    # Reset index and rename columns
    df.reset_index(inplace=True)
    df.rename(columns={'index': 'Date'}, inplace=True)
    return df.head(1000)


def realgdppercapita(key):
    url = f'https://www.alphavantage.co/query?function=REAL_GDP_PER_CAPITA&apikey={key}'
    r = requests.get(url)
    data = r.json()
    time_series_data = data["data"]
    # Convert to DataFrame
    df = pd.DataFrame(time_series_data).T
    # Reset index and rename columns
    df.reset_index(inplace=True)
    df.rename(columns={'index': 'Date'}, inplace=True)
    return df
    

def inflation(key):
    url = f'https://www.alphavantage.co/query?function=INFLATION&apikey={key}'
    r = requests.get(url)
    data = r.json()
     time_series_data = data["data"]
    # Convert to DataFrame
    df = pd.DataFrame(time_series_data).T
    # Reset index and rename columns
    df.reset_index(inplace=True)
    df.rename(columns={'index': 'Date'}, inplace=True)
    return df


def process_and_save_data(tickers):  #need to get list of tickers from gcs/bq
    """
    Runs each ticker through the list of functions, processes the resulting DataFrames,
    and saves them as CSV files.

    Parameters:
        tickers (list): List of ticker symbols. 
        api_keys (list): List of API keys corresponding to each function.

    Returns:
        None
    """

    api_keys = ['L0S0FYJGSKZLWC8R', '63TMPZD2NJJ51WFA', 'YBATNF7ZMC9EJ246']
    for i in range(len(tickers)):
        ticker = tickers[i]
        key = api_keys[i]
        
        # Call the 'cp' function and save the DataFrame as a CSV file
        df_cp = cp(ticker, key)
        filename_cp = f"{ticker}_cp.csv"
        df_cp.to_csv(filename_cp, index=False)
        print(f"DataFrame for {ticker} with 'cp' saved as {filename_cp}")
        
        # Call the 'rsi' function and save the DataFrame as a CSV file
        df_rsi = rsi(ticker, key)
        filename_rsi = f"{ticker}_rsi.csv"
        df_rsi.to_csv(filename_rsi, index=False)
        print(f"DataFrame for {ticker} with 'rsi' saved as {filename_rsi}")
        
        # Call the 'sma' function and save the DataFrame as a CSV file
        df_sma = sma(ticker, key)
        filename_sma = f"{ticker}_sma.csv"
        df_sma.to_csv(filename_sma, index=False)
        print(f"DataFrame for {ticker} with 'sma' saved as {filename_sma}")

        # Call the 'eps' function and save the DataFrame as a CSV file
        df_eps = eps(ticker, key)
        filename_eps = f"{ticker}_eps.csv"
        df_eps.to_csv(filename_eps, index=False)
        print(f"DataFrame for {ticker} with 'eps' saved as {filename_sma}")

    api_keys2 = ['L0S0FYJGSKZLWC8R', '63TMPZD2NJJ51WFA', 'YBATNF7ZMC9EJ246'] #to pull fundamental market data
    functions = [dty, realgdppercapita, inflation]
    filenames = ['treasury_yield.csv', 'real_gdp_per_capita.csv', 'inflation.csv']
    
    for i in range(len(api_keys2)):
        key = api_keys2[i]
        func = functions[i]
        filename = filenames[i]
        
        df = func(key)
        df.to_csv(filename, index=False)
        print(f"DataFrame saved as {filename}")
    

    

# Example usage:
#tickers = ['AAPL', 'MSFT', 'GOOGL']  # Example list of tickers


process_and_save_data(tickers, api_keys)